In [1]:
import sqlite3

In [2]:
with sqlite3.connect("scrabble.db") as conn:
    conn.execute("""
        CREATE TABLE players (
            player_id INTEGER PRIMARY KEY,
            name TEXT NOT NULL
        )
    """)

In [3]:
with sqlite3.connect("scrabble.db") as conn:
    conn.execute("""
        CREATE TABLE games (
            game_id INTEGER PRIMARY KEY,
            started_by INTEGER,
            winner INTEGER,
            FOREIGN KEY (started_by) REFERENCES players(player_id),
            FOREIGN KEY (winner) REFERENCES players(player_id)
        )
    """)

In [4]:
with sqlite3.connect("scrabble.db") as conn:
    conn.execute("""
        CREATE TABLE game_scores (
            game_id INTEGER,
            player_id INTEGER,
            score INTEGER,
            PRIMARY KEY (game_id, player_id),
            FOREIGN KEY (game_id) REFERENCES games(game_id),
            FOREIGN KEY (player_id) REFERENCES players(player_id)
        )
    """)

In [5]:
with sqlite3.connect("scrabble.db") as conn:
    conn.execute("""
    CREATE TABLE game_moves (
        game_id INTEGER,
        player_id INTEGER,
        move_number INTEGER,
        points INTEGER,
        PRIMARY KEY (game_id, player_id, move_number),
        FOREIGN KEY (game_id) REFERENCES games(game_id),
        FOREIGN KEY (player_id) REFERENCES players(player_id)
    )
    """)

In [6]:
with sqlite3.connect("scrabble.db") as conn:
    conn.execute("""
        INSERT INTO players (name)
        VALUES ('Deniz'), ('Danyel'), ('Robin')
    """)

In [7]:
def get_player_id(connection, name: str) -> int | None:
    response = connection.execute("""
        SELECT player_id
        FROM players
        WHERE name = ?
    """, (name,))

    result = response.fetchone()
    if result:
        return result[0]

    return None

In [8]:
import pandas as pd

df = pd.read_csv("scrabble_data.csv")

with sqlite3.connect("scrabble.db") as conn:
    deniz_id = get_player_id(conn, "Deniz")
    danyel_id = get_player_id(conn, "Danyel")
    robin_id = get_player_id(conn, "Robin")

with sqlite3.connect("scrabble.db") as conn:
    for _, row in df.iterrows():
        started_by_id = get_player_id(conn, row["starting_player"])
        winner_id = get_player_id(conn, row["winner"])

        cursor = conn.execute("""
            INSERT INTO games (started_by, winner)
            VALUES (?, ?)
        """, (started_by_id, winner_id))

        game_id = cursor.lastrowid

        conn.executemany("""
            INSERT INTO game_scores (game_id, player_id, score)
            VALUES (?, ?, ?)
        """, [
            (game_id, deniz_id, row["Deniz"]),
            (game_id, danyel_id, row["Danyel"]),
            (game_id, robin_id, row["Robin"])
        ])

In [9]:
import pandas as pd

df = pd.read_csv("scrabble_data.csv")

df


,Deniz,Danyel,Robin,winner,starting_player,deniz_scores,danyel_scores,robin_scores
0,195,181,229,Robin,NaN,NaN,NaN,NaN
1,92,123,195,Robin,NaN,NaN,NaN,NaN
2,155,150,136,Deniz,NaN,NaN,NaN,NaN
3,127,67,195,Robin,NaN,NaN,NaN,NaN
4,146,208,165,Danyel,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
107,214,192,152,Deniz,Robin,NaN,NaN,NaN
108,154,203,224,Robin,Robin,NaN,NaN,NaN
109,194,209,144,Danyel,Deniz,NaN,NaN,NaN
110,166,140,203,Robin,Deniz,NaN,NaN,NaN


In [14]:
with sqlite3.connect("scrabble.db") as conn:
    response = conn.execute("""
        SELECT * FROM game_scores
    """)
    result = response.fetchall()

    print(result)

[(1, 1, 195), (1, 2, 181), (1, 3, 229), (2, 1, 92), (2, 2, 123), (2, 3, 195), (3, 1, 155), (3, 2, 150), (3, 3, 136), (4, 1, 127), (4, 2, 67), (4, 3, 195), (5, 1, 146), (5, 2, 208), (5, 3, 165), (6, 1, 142), (6, 2, 246), (6, 3, 238), (7, 1, 222), (7, 2, 159), (7, 3, 133), (8, 1, 79), (8, 2, 165), (8, 3, 202), (9, 1, 162), (9, 2, 168), (9, 3, 239), (10, 1, 154), (10, 2, 199), (10, 3, 186), (11, 1, 196), (11, 2, 229), (11, 3, 87), (12, 1, 91), (12, 2, 169), (12, 3, 170), (13, 1, 156), (13, 2, 222), (13, 3, 136), (14, 1, 137), (14, 2, 202), (14, 3, 120), (15, 1, 202), (15, 2, 185), (15, 3, 183), (16, 1, 153), (16, 2, 164), (16, 3, 143), (17, 1, 202), (17, 2, 171), (17, 3, 140), (18, 1, 108), (18, 2, 139), (18, 3, 196), (19, 1, 127), (19, 2, 150), (19, 3, 170), (20, 1, 164), (20, 2, 117), (20, 3, 186), (21, 1, 217), (21, 2, 147), (21, 3, 78), (22, 1, 161), (22, 2, 152), (22, 3, 152), (23, 1, 185), (23, 2, 164), (23, 3, 207), (24, 1, 171), (24, 2, 132), (24, 3, 246), (25, 1, 179), (25, 2, 18

In [22]:
with sqlite3.connect("scrabble.db") as conn:
    conn.row_factory = sqlite3.Row
    response = conn.execute("""
        SELECT 
            g.game_id, 
            sp.name AS started_by, 
            wp.name AS winner,
            p.name AS player_name,
            gs.score
        FROM games g
        LEFT JOIN players sp ON sp.player_id = g.started_by
        LEFT JOIN players wp ON wp.player_id = g.winner
        INNER JOIN game_scores gs ON gs.game_id = g.game_id
        INNER JOIN players p ON p.player_id = gs.player_id
    """)

    result = [dict(row) for row in response.fetchall()]
    result = result[::-1]
    for r in result:
        print(r)


{'game_id': 112, 'started_by': 'Danyel', 'winner': 'Robin', 'player_name': 'Robin', 'score': 208}
{'game_id': 112, 'started_by': 'Danyel', 'winner': 'Robin', 'player_name': 'Danyel', 'score': 180}
{'game_id': 112, 'started_by': 'Danyel', 'winner': 'Robin', 'player_name': 'Deniz', 'score': 204}
{'game_id': 111, 'started_by': 'Deniz', 'winner': 'Robin', 'player_name': 'Robin', 'score': 203}
{'game_id': 111, 'started_by': 'Deniz', 'winner': 'Robin', 'player_name': 'Danyel', 'score': 140}
{'game_id': 111, 'started_by': 'Deniz', 'winner': 'Robin', 'player_name': 'Deniz', 'score': 166}
{'game_id': 110, 'started_by': 'Deniz', 'winner': 'Danyel', 'player_name': 'Robin', 'score': 144}
{'game_id': 110, 'started_by': 'Deniz', 'winner': 'Danyel', 'player_name': 'Danyel', 'score': 209}
{'game_id': 110, 'started_by': 'Deniz', 'winner': 'Danyel', 'player_name': 'Deniz', 'score': 194}
{'game_id': 109, 'started_by': 'Robin', 'winner': 'Robin', 'player_name': 'Robin', 'score': 224}
{'game_id': 109, 'sta

1. Table of all games (scores of each player, start player)
    1a. Moves of a game (evolution of points over time)
2. Stats for each player (best game, worst game, average points, ...)